# Finviz Analytics


### What is Finviz?
FinViz aims to make market information accessible and provides a lot of data in visual snapshots, allowing traders and investors to quickly find the stock, future or forex pair they are looking for. The site provides advanced screeners, market maps, analysis, comparative tools and charts.

### Why?
Leverage NRT financial stats to create custom stock screens and perspectives to identify value with in volatile market conditions.

***
## Prerequisites 

### finviz
Finviz is a stock screener and trading tool used for creating financial displays. Professional traders frequently use this platform to save time because Finviz allows traders and investors to quickly screen and find stocks based on set criteria.


### pandas, pandas_profiling
Pandas needs no introduction.  Pandas_profiling creates beautiful html data profiles.

### nest_asyncio
Nest_asyncio supports asynchronous call for use with an interactive broker.



In [1]:
import pandas as pd 
import logging 
from finviz.screener import Screener

log = logging.getLogger() 
console = logging.StreamHandler()
format_str = '%(asctime)s\t%(levelname)s -- %(processName)s -- %(message)s'
console.setFormatter(logging.Formatter(format_str))
log.addHandler(console) 
log.setLevel(logging.INFO) 


***
## Load environment and runtime variables



In [6]:
'''
MODULE is used to identify and segment runtime and environment variables from config files.
'''
MODULE = 'ldg_finviz'


'''
Load configuration fiiles from \config.  Instansiate variables with config file names.  
'''
import os 
d = os.getcwd()
df = d + '\\config\\'
try: 
    for i in os.listdir(df):
        k = i[:-4] 
        v = open(df + i).read()
        v = eval(v)
        exec("%s=%s" % (k,v))   
        log.info('loaded: ' + k)
except:
    log.error('issue encountered with eval(data): ' + str(v))


2020-06-19 11:01:11,923	INFO -- MainProcess -- loaded: api_params
2020-06-19 11:01:11,924	INFO -- MainProcess -- loaded: data_var
2020-06-19 11:01:11,925	INFO -- MainProcess -- loaded: env_var
2020-06-19 11:01:11,926	INFO -- MainProcess -- loaded: err_var
2020-06-19 11:01:11,927	INFO -- MainProcess -- loaded: map_generic_fn
2020-06-19 11:01:11,927	INFO -- MainProcess -- loaded: map_landing_dataset_code
2020-06-19 11:01:11,929	INFO -- MainProcess -- loaded: transform


In [7]:
def get_data_finviz(generate_data_profile=False):
    '''
    Download FinViz 15min delayed stock data.  
    * filter - Filter stock universe using filters variable.  
    * Select datasets to download using the map_api_fv_table_key.config  
    * Dataset options include: 
        'Overview': '111',
        'Valuation': '121',
        'Ownership': '131',
        'Performance': '141',
        'Custom': '152',
        'Financial': '161',
        'Technical': '171'
    * Refer to /docs for dataset details.
    
    Output data in .csv format to landing.
    '''
    import nest_asyncio
    nest_asyncio.apply()
    
    #load variables
    ldg_path = env_var.get('ldg_path') 
    filters = api_params['filter']

    #loop through datasets to download from Screener & write to file.
    for i in api_params['datasets']:
        log.info('downloading:' + i.get('dataset').lower())
        stock_list = Screener(filters=filters, table= i.get('dataset'))
        stock_list.to_csv(ldg_path + 'stock_screener_' +  i.get('dataset').lower() + '.csv')
    
    if generate_data_profile == True:
        log.info('begin pandas profile.html generation')
        generate_docs('ldg_path')

def get_transform(target):
    '''
    Get transform from transform.cfg for target dataset
    #returns a list of dicts with transform logic in format {field:value, fn:value} 
    '''

    lst = [i.get(target) for i in transform['root'] if i.get(target) != None]
    if lst != []: lst = lst[0]

    return lst

def apply_transform(df, transform, target):
    '''
    apply list of tranformstions to dataframe
    '''
    import numpy as np
    log.info('begin transforms for: ' + target)
    
    try: 
        for t in transform: 

            #get function to apply
            fn = t.get('fn')    
            #get reusable function from map_generic_fn if fn starts with $ 
            if fn[0] == '$': fn = map_generic_fn.get(fn)

            #get field or fields to update
            field = t.get('field')
            if field[0] == '[':field = eval(field) 
            #log.info('apply transform: {field, function} ' + str(field) + ' ' + fn)

            #apply transformation
            df[field] = eval(fn)
    except: 
        log.error('error encountered with table:' + str(target) + ' field:' + str(field) + ' fn:' + str(fn) )
    
    log.info('end transforms for: ' + target)
    return 

def normalize_data():
    '''
    Perform preprocessing and copy data to staging area.  
    Preprocessing steps are included in transform.cfg and typically include:
    - metadata validation / data contract
    - preliminary schema normalization
    - data type validation & associated cleansing.  
    '''
    
    ldg_path = env_var.get('ldg_path') 
    stg_path = env_var.get('stg_path')
    
    try:
        #for each dataset in map_landing_dataset_code
        for i in map_landing_dataset_code.get(MODULE):

            #load file for meta contract and data type conversion
            file = ldg_path + i.get('file')
            code = i.get('code')
            df = pd.read_csv(file)

            trns = get_transform(code)
            if trns != []:
                #apply transformation to dataframe
                apply_transform(df,trns,code)

            df.to_csv(stg_path + i.get('file'))
            #TODO Normalize column names, tolwower() with underscores...             
    except:
        log.error('error in normalize_data()')
        
def generate_docs(path):
    '''
    generate pandas_profile.html reports
    '''
    from pandas_profiling import ProfileReport
    
    data_path = env_var.get(path)
    
    try: 
        for i in map_landing_dataset_code[MODULE]:
            file = i.get('file')
            df = pd.read_csv(data_path + file)
            profile = ProfileReport(df, title= 'Profile: ' + file + ' (Landing)')
            profile.to_file(data_path + 'profile_' + file[0:-4] + '.html')
    except:
        log.error('error in generating pandas_profile.html')

def convert_unit(u):
    if len(u) != 1: 
        ua = u
        u = str(u[-1])
        val = str(ua[0:-1]).replace('.','')
    else:  val = ''
    u=u.lower()

    if u == 'm':
        val += '0000'
    elif u == 'b':
        val += '0000000'
    return val 

# Download & stage data 

In [8]:
log.info('begin downloading finviz')
#get_data_finviz(generate_data_profile=False)
log.info('end downloading finviz')


log.info('begin finviz preprocessing')
normalize_data()
log.info('end finviz preprocessing')

2020-06-19 11:01:17,476	INFO -- MainProcess -- begin downloading finviz
2020-06-19 11:01:17,477	INFO -- MainProcess -- end downloading finviz
2020-06-19 11:01:17,478	INFO -- MainProcess -- begin finviz preprocessing
2020-06-19 11:01:17,507	INFO -- MainProcess -- begin transforms for: ssfin
2020-06-19 11:01:17,515	INFO -- MainProcess -- Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2020-06-19 11:01:17,515	INFO -- MainProcess -- NumExpr defaulting to 8 threads.
2020-06-19 11:01:17,607	INFO -- MainProcess -- end transforms for: ssfin
2020-06-19 11:01:17,708	INFO -- MainProcess -- begin transforms for: ssovw
2020-06-19 11:01:17,729	INFO -- MainProcess -- end transforms for: ssovw
2020-06-19 11:01:17,794	INFO -- MainProcess -- begin transforms for: sstec
2020-06-19 11:01:17,892	INFO -- MainProcess -- end transforms for: sstec
2020-06-19 11:01:18,001	INFO -- MainProcess -- begin transforms for: sscust
2020-06-19 11:01:18,023	INFO -- MainProc

In [9]:
def enrich_data_finviz():
    import numpy as np
    
    ENF = err_var.get('no_param')
    stg_path = env_var.get('stg_path')
    
    fown =stg_path + data_var[MODULE].get('stock_ownership',ENF)
    fovr =stg_path + data_var[MODULE].get('stock_overview',ENF)
    key = data_var[MODULE].get('stock_key',ENF)
    
    if (fown==ENF) or (fovr== ENF) or (key == ENF):
        e = 'missing file or key name'
        log_diagnostics('enrich_data_finviz',e,env_var)
        return
    
    #generate additional attibutes
    df_own = pd.read_csv(fown).reset_index()
    df_own.set_index(key,inplace=True)
    df_view = pd.read_csv(fovr).reset_index()
    df_view.set_index(key,inplace=True)
    df = pd.merge(df_own,df_view,how='inner',left_index = True, right_index=True)
    df.reset_index(inplace=True)

    target = 'stg_finviz_summary'            
    transform = get_transform(target) 
    apply_transform(df,transform,target)
    df = df[['Ticker','eps','earnings','P/E','e/p','Outstanding']]

    df.to_csv(stg_path + 'stock_screener_summary.csv')


enrich_data_finviz()

2020-06-19 11:01:28,134	INFO -- MainProcess -- begin transforms for: stg_finviz_summary
2020-06-19 11:01:28,142	INFO -- MainProcess -- end transforms for: stg_finviz_summary
